In [ ]:
import sys
import re
import os
import time
from tqdm import tqdm

import numpy as np
# import pandas as pd
from bs4 import BeautifulSoup
import jieba #分词
from wordcloud import WordCloud #词云
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['font.family'] = 'simhei'
import random
import pandas as pd
#from PIL import Image
import csv

## 配置一些函数

In [ ]:
# sens_words_path = 'sensitive_words.txt'
# sens_words = read_words_list(sens_words_path)
# sens_words = list(set(sens_words))
# sens_words.sort()
# sens_words_str = "\n".join(sens_words)
# fh = open('sensitive_words.txt','w')
# fh.write(sens_words_str)
# fh.close()

In [ ]:
#读取词表
def read_words_list(path):
    with open(path) as f:
        lines = f.readlines()
    
    strlist = []
    for l in lines:
        if '#' != l[0] and '' != l.strip():
            l = l.strip()
            strlist.append(l)
    return strlist

#查找敏感词
def check_sens_words(text, sens_words):
    ttext = text.strip()
    sw_dict = {} #敏感词
    for sw in sens_words:
        n = ttext.count(sw) #敏感词出现次数
        if n>0:
            if not sw_dict.__contains__(sw):
                sw_dict[sw] = 0
            sw_dict[sw] += n
    return sw_dict

def sentence_cut(text):
    pattern = pattern = r'\.|/|;|\?|!|。|；|！|……|\n'
    pre_sentence = re.split(pattern, text)
    sentence = []
    for pre in pre_sentence:
        if len(pre) > 8:
            sentence.append(pre)
    return sentence

In [ ]:
#提取ao3文章
def extract_ao3_work(html):
    soup = BeautifulSoup(html, 'html.parser')
    #提取分级标签
    rating_dd = soup.find('dd', attrs={'class': 'rating tags'}) #找到分级标签
    rating_a = rating_dd.find('a', attrs={'class': 'tag'}) #找到对应的a标签
    rating = rating_a.string #获得标签文字
    
    stats_dd = soup.find('dl', attrs={'class': 'stats'})
    #提取点击量
    hits_dd = stats_dd.find('dd', attrs={'class': 'hits'}) #找到分级标签
    try:
        hits = int(hits_dd.string)
    except AttributeError:
        hits = 0
    
    #提取发布日期
    published_dd = stats_dd.find('dd', attrs={'class': 'published'}) #找到分级标签
    date_str = published_dd.string

    #提取文章
    article_div = soup.find('div', attrs={'role': 'article'}) #找到文章标签
    article_userstuff = article_div.find('div', attrs={'class': 'userstuff'})
    article_str = str(article_userstuff)
    article_str = article_str.replace("<br>","")
    article_str = article_str.replace("<p>","")
    article_str = re.sub("<div[a-z\ 0-9=\"\',\[\]\-\_]*[ \>\n]","",article_str)
    article_str = re.sub("<p[a-z\ 0-9=\"\',\[\]\-\_]*[ \>\n]","",article_str)
    article_str = re.sub("<br[a-z\ 0-9=\"\',\[\]\-\_]*[ \>\n]","",article_str)
    article_str = article_str.replace("</p>","。")
    article_str = article_str.replace("</br>","\n")
    article_str = article_str.replace("&nbsp;"," ")
    article_str = article_str.replace("</div>","")
    article_str = re.sub("<b[a-z\ 0-9=\"\',\[\]\-\_]*[ \>\n]","",article_str)
    article_str = article_str.replace("<\b>","")
    article_str = re.sub("<img[a-z\ 0-9=\"\',\[\]\-\_]*[ \>\n]","",article_str)
    article_str = article_str.replace("> ","")
    article_str = article_str.replace("<br","")
    article_str = article_str.replace("<em","") #漏网
    article_str = re.sub("[a-z\ 0-9=\"\',\[\]\-\_]*\"\>","",article_str)
    
    # print(article_str)
    # time.sleep(3)
    return rating, hits, date_str, article_str

In [ ]:
sens_words_path = 'sensitive_words.txt'
sens_words = read_words_list(sens_words_path)
base_path = "fulltext/"
sex_path = "train/sex/"
pos_path = "train/pos/"

In [ ]:
ao3_pbar = tqdm(os.listdir(base_path))

In [ ]:
#爬sex句子
for work in ao3_pbar:
    work_path = os.path.join(base_path,work)
    with open(work_path) as f:
        work_str = f.read() #读取文章
    rating, hits, date_str, article_str = extract_ao3_work(work_str)
#     #不要低分级文章
#     if "General Audiences" in rating or "Teen And Up Audiences" in rating:
#         continue
#     ao3_pbar.set_description("{} | {}".format(work, rating))
#     print(work, rating)
#     print('-'*40)
    sents = sentence_cut(article_str)
    for it,sent in enumerate(sents):
        sens_pkg = check_sens_words(sent, sens_words)
        if sens_pkg:
#             sents_buf = [sent]
            sum_sens = 0 #
            for spv in sens_pkg.values(): #统计词汇数量
                sum_sens += spv
            prev_sum_sens = sum_sens-2
            #增加涉及敏感词的句子量
            final_subi = 0
            stop_flg = False
            for subi in range(1,random.randint(2,6)):
                final_subi = subi
                if prev_sum_sens > sum_sens-2: #增加量小于2
                    stop_flg = True
                    break
                prev_sum_sens = sum_sens
                if it-subi>=0:
                    sent = sents[it-subi]+ "。" + sent
#                     sents_buf.append(sents[it-subi]) #加入句子
                if it+subi<len(sents):
                    sent += "。" + sents[it+subi]
#                     sents_buf.append(sents[it+subi]) #加入句子
                sens_pkg = check_sens_words(sent, sens_words) #检测敏感词
                sum_sens = 0
                for spv in sens_pkg.values(): #统计词汇数量
                    sum_sens += spv
            
#             #果然敏感词很多，则包含前后普通句子
#             if sum_sens>5:
#                 if not stop_flg:
#                     final_subi += 1
#                 if it-final_subi>=0:
# #                     sent = sents[it-final_subi]+ "。" + sent
#                     sents_buf.append(sents[it-final_subi]) #加入句子
#                 if it+final_subi<len(sents):
# #                     sent += "。" + sents[it+final_subi]
#                     sents_buf.append(sents[it+final_subi]) #加入句子
#                 final_subi += 1
            if sum_sens>1:
#                 print(sent, sens_pkg, final_subi)
                sent_path = os.path.join(sex_path, work.replace('.html',"_")+str(it)+'.txt')
                fh = open(sent_path,'w')
                fh.write(sent)
                fh.close()
#                 for t,s in enumerate(sents_buf):
#                     sent_path = os.path.join(sex_path, work.replace('.html',"_")+str(it) +"_"+str(t)+'.txt')
#                     fh = open(sent_path,'w')
#                     fh.write(s)
#                     fh.close()
#     print('-'*40)

In [ ]:
#中文比例
def str_ratio_zh(instr):
    cnt = 0
    for s in instr:
        # 中文字符范围
        if '\u4e00' <= s <= '\u9fa5':
            cnt += 1
    return cnt/len(instr)

In [ ]:
#爬pos句子
for work in ao3_pbar:
    work_path = os.path.join(base_path,work)
    with open(work_path) as f:
        work_str = f.read() #读取文章
    rating, hits, date_str, article_str = extract_ao3_work(work_str)
    #要低分级文章
    skip_rate = 40
    if "General Audiences" not in rating and "Teen And Up Audiences" not in rating:
        skip_rate = 60
#     ao3_pbar.set_description("{} | {}".format(work, rating))
#     print(work, rating)
#     print('-'*40)
    sents = sentence_cut(article_str)
    final_it = -1
    for it,sent in enumerate(sents):
        #抛弃一定比例的句子
        if it < 5 or random.randint(0,100) < skip_rate or it<final_it or str_ratio_zh(sents)<0.8 or it > len(sents)-5:
            continue
            
        sens_break_flg = True
        sens_pkg = check_sens_words(sent, sens_words)
        if not sens_pkg:
#             sents_buf = [sent]
            sens_break_flg = False
            #增加涉及敏感词的句子量
            for subi in range(1,random.randint(2,6)):
                if it-subi>=0:
                    sent = sents[it-subi]+ "。" + sent
#                     sents_buf.append(sents[it-subi])
                if it+subi<len(sents):
                    sent += "。" + sents[it+subi]
#                     sents_buf.append(sents[it+subi])
                    final_it = it+subi
                    
                sens_pkg = check_sens_words(sent, sens_words) #检测敏感词
                if sens_pkg:
                    sens_break_flg = True
                    break
            if not sens_break_flg: #不涉sex文抛弃
                #print(sent, sens_pkg, final_subi)
                sent_path = os.path.join(pos_path, work.replace('.html',"_")+str(it)+'.txt')
                fh = open(sent_path,'w')
                fh.write(sent)
                fh.close()
#                 for t,s in enumerate(sents_buf):
#                     sent_path = os.path.join(pos_path, work.replace('.html',"_")+str(it) +"_"+str(t)+'.txt')
#                     fh = open(sent_path,'w')
#                     fh.write(s)
#                     fh.close()
            else:
                final_it += 3
            final_it += 1
#     print('-'*40)

In [ ]:
# csv_path = "train/ao3.csv"
csv_train_path = "train/train.csv"
csv_test_path = "train/test.csv"
#label content
ao3sents = []

train_num = 78000
max_pos_num = 125450

sex_train_pbar = tqdm(os.listdir(sex_path)[:train_num])
pos_train_pbar = tqdm(os.listdir(pos_path)[:int(1.025*train_num)])

sex_test_pbar = tqdm(os.listdir(sex_path)[train_num:])
pos_test_pbar = tqdm(os.listdir(pos_path)[int(1.03*train_num):])

with open(csv_train_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for sex in sex_train_pbar:
        work_path = os.path.join(sex_path,sex)
        with open(work_path) as fh:
            sent = fh.read()
        writer.writerow([1, sent])
    for pos in pos_train_pbar:
        work_path = os.path.join(pos_path,pos)
        with open(work_path) as fh:
            sent = fh.read()
        writer.writerow([0, sent])
        
with open(csv_test_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for sex in sex_test_pbar:
        work_path = os.path.join(sex_path,sex)
        with open(work_path) as fh:
            sent = fh.read()
        writer.writerow([1, sent])
    for pos in pos_test_pbar:
        work_path = os.path.join(pos_path,pos)
        with open(work_path) as fh:
            sent = fh.read()
        writer.writerow([0, sent])